# 네이버 뉴스 크롤러 만들기 연습
아래 코드를 실행해야 이하부분이 돌아갑니다.

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 제출 평가 부분

In [ ]:
def main_crawling(query, start_date, end_date, sort_type, max_page):
    '''
    함수기능
    input args::
        query: 설명
        return:
        아웃풋 형태 설명
    '''
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2021.01.01'
    if len(end_date) != 10:
        end_date = '2021.12.31'
    if sort_type not in ['0', '1', '2']:
        sort_type = '0'
    if max_page == '':
        max_page = 5


    # 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
    categories = []
    titles = []
    dates = []
    articles = []
    article_urls = []
    press_companies = []

    # 주어진 일자를 쿼리에 맞는 형태로 변경해줍니다.
    start_date = start_date.replace(".", "")
    end_date = end_date.replace(".", "")

    # 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
    current_call = 1
    last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당


    while current_call <= last_call:

        print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

        url = "https://search.naver.com/search.naver?where=news&query=" + query \
              + "&sort=" + sort_type \
              + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
              + "to" + end_date \
              + "%2Ca%3A&start=" + str(current_call)

        urls_list = []
        try: # 네이버 뉴스 검색결과 페이지 자체에 접근이 불가능할 경우 에러가 발생할 수 있습니다.
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')

            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    urls_list.append(urls["href"])
        except:
            print('해당 뉴스 검색 페이지의 네이버 뉴스 링크를 모으는 중 에러가 발생했습니다. : ', url)

        # urls_list : 해당 페이지에 있는 "네이버 뉴스"의 링크 모음(list)
        if urls_list != []:
            for url in urls_list:
                try: # 특정 뉴스 기사글 하나를 크롤링하는 중 에러가 발생할 수 있습니다.ㄴ
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news, 'html.parser')
                    # 1) 카테고리 class="media_end_categorize_item"
                    category = source_news.find('em',{'class' : 'media_end_categorize_item'}).get_text()

                    title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
                    print('Processing article : {}'.format(title))

                    date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                    article = source_news.find('article', {'id' : 'dic_area'}).get_text()
                    article = article.replace("\n", "")
                    article = article.replace("\'", "")
                    article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                    article = article.replace("동영상 뉴스       ", "")
                    article = article.replace("동영상 뉴스", "")
                    article = article.strip()

                    press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()

                    categories.append(category)
                    titles.append(title)
                    dates.append(date)
                    articles.append(article)
                    press_companies.append(press_company)
                    article_urls.append(url)

                    time.sleep(1)
                except:
                    print('\n*** {}번부터 {}번까지의 기사글을 크롤링하는 중 문제가 발생했습니다.'.format(current_call, current_call+9))
                    print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))

        else:
            pass

        current_call += 10


    article_df = pd.DataFrame({'date':dates,
                               'category':categories,
                               'press':press_companies,
                               'title':titles,
                               'document':articles,
                               'link':article_urls,
                               })

    # article_df.to_csv('result_{}.csv'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

    print('\n크롤링이 성공적으로 완료되었습니다!')
    print('\n크롤링 결과를 다음 파일에 저장하였습니다 : {}'.format(datetime.now().strftime('%y%m%d_%H%M')))

    return article_df

In [ ]:
query = input('검색어를 입력해주세요. (ex. 데이터 분석) : ')
start_date = input('검색 시작 날짜를 입력해주세요. (형식 : 2021.01.01) : ')
end_date = input('검색 종료 날짜를 입력해주세요. (형식 : 2021.12.31) : ')
sort_type = input('정렬 타입을 입력해주세요 (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : ')
max_page = input('크롤링을 원하는 전체 페이지 수를 입력해주세요. (ex. 5) : ')

if start_date > end_date:
    print('\n시작 날짜는 종료 날짜보다 이후로 지정하실 수 없습니다. 다시 실행해주세요!')
elif max_page == '':
    max_page = 5
    print('\n원하시는 페이지 수가 입력되지 않았습니다. 5 페이지까지만 크롤링을 진행합니다.')
    main_crawling(query, start_date, end_date, sort_type, max_page)
else:
    max_page = int(max_page)
    scraped_news = main_crawling(query, start_date, end_date, sort_type, max_page)
    scraped_news.to_csv('/content/drive/MyDrive/data/result_{}.csv'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

검색어를 입력해주세요. (ex. 데이터 분석) : 부동산
검색 시작 날짜를 입력해주세요. (형식 : 2021.01.01) : 2024.01.05
검색 종료 날짜를 입력해주세요. (형식 : 2021.12.31) : 2024.02.05
정렬 타입을 입력해주세요 (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : 0
크롤링을 원하는 전체 페이지 수를 입력해주세요. (ex. 5) : 50

1번째 기사글부터 크롤링을 시작합니다.
Processing article : 집값 4억 '뚝'…부동산 거래 침체에 노도강 직격탄
Processing article : 한덕수 국무총리 "부동산 PF집중 관리하겠다"
Processing article : "올해 하반기 사이클 최저…부동산 투자 늘려라"
Processing article : 2024 대구 부동산 전망…올해 2만가구 입주 소화될까?
Processing article : “車 할부금융 카드사에 뺏겨”… 중소캐피탈이 부동산 PF에 손댄 이유
Processing article : 中인민은행 등, 부동산 침체 속 임대주택 시장 금융지원 약속
Processing article : 부동산통계조작 혐의 전 차관 등 영장기각, 검찰 "납득불가"
Processing article : “이정도면 아이 낳겠습니까?” 5억 파격 대출에 7만 가구 특공 나온다 [부동산360]
Processing article : [단독] 명지학원 폐교위기 탈출…"부동산 등 매각 1700억 채무 변제"
Processing article : ‘부동산 PF발 위기’ 건설사 옥석 가리기 본격화…“재개발 사업 리스크는 제한적”

11번째 기사글부터 크롤링을 시작합니다.
Processing article : 증권사, 부동산 PF 위기에도 실적 선방으로 극복?
Processing article : 2024 대구 부동산 전망…올해 2만가구 입주 소화될까?
Processing article : “車 할부금융 카드사에 뺏겨”… 중소캐피탈이 부동산 PF에 손댄 이유
Proce